# Capstone Project - The Battle of Neighbourhoods - City of Toronto

Applied Data Science Capstone from IBM (on Coursera.org)

## 1. Introduction

### 1.1. Background

In "Applied Data Science Capstone" course, we explored, segmented, and clustered the neighbourhoods in the city of Toronto based on venues in each postcode area.

This capstone project will continue the same analysis with additional features (such as elementary and secondary schools' ratings, 2016 census data from Statistic Canada, and housing data from HouseSigma) to better describe the characteristics of each neighbourhood.

### 1.2. Problem

There are many factors that could characterize a neighbourhood besides venues or interest points.

Other factors could include, but not limited to, house and rental pricing, availability of good elementary and secondary schools for the children, average family incomes for selection reference.

Due to time limitation, only detached houses will be considered in this project.

This analysis tries to answer the question: Which neighbourhoods have the similar features  from where a family with young children can purchase a detached residential property?

### 1.3. Interest

Families with young children of school age who are looking for a residential property in City of Toronto.

Note: _Residential property or house_ referring here could be a detached house, semi-detached house, townhouse, or apartment.

## 2. Data Acquisition and Cleaning

### 2.1. FSA Postcodes of City of Toronto
[https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

HTML scraping to get all Forward Sortation Area (FSA) postcodes, boroughs, and neighbourhoods in City of Toronto.

These data cleaning steps will be applied:
- The dataframe will consist of three columns: Postcode, Borough, and Neighbourhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighbourhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighbourhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighbourhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighbourhood, then the neighbourhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighbourhood columns will be Queen's Park.

Note: The FSA postcodes are the first three characters of Canadian postal codes.  For example, _M5H_ is the FSA postcode for the postal code '_M5H_ 2N2' (where Toronto City Hall is located).

In addition, the latitude and longitudes coordinates of each FSA postcode will be loaded from [http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv](http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv) file.

### 2.2. Foursquare Location Data

Foursquare API will be used to find out the following in a postcode area:
- Venues: Limited to maximum 100 venues
- Schools: School ranking average in 5000m (5km)
    * Elementary School: 4f4533804b9074f6e4fb0105
    * High School: 4bf58dd8d48988d13d941735
    * Middle School: 4f4533814b9074f6e4fb0106
    * Private School: 52e81612bcbc57f1066b7a46
- Transportation: How many stations or stops in 800m (10 min walking distance)
    * Bus Station: 4bf58dd8d48988d1fe931735
    * Bus Stop: 52f2ab2ebcbc57f1066b8b4f
    * Train Station: 4bf58dd8d48988d129951735
    * Transportation Service: 54541b70498ea6ccd0204bff

Above listed Foursquare venue category ids can be found at [https://developer.foursquare.com/docs/resources/categories](https://developer.foursquare.com/docs/resources/categories)

### 2.3. Fraser Institute - School Rating

HTML scraping to find out school ratings:
- Elementary: [http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx?schooltype=elementary](http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx?schooltype=elementary)
- Secondary: [http://ontario.compareschoolrankings.org/secondary/SchoolsByRankLocationName.aspx?schooltype=secondary](http://ontario.compareschoolrankings.org/secondary/SchoolsByRankLocationName.aspx?schooltype=secondary)

School ratings are between 0 and 10.  Division by 10 (ten) will be needed to normalize school ratings for clustering.

We will ignore schools without school ratings from this data source and we won't take into account school boundaries.

### 2.4. HouseSigma - Median Price Report

[https://housesigma.com/site/en/analyze/city-median-price-report](https://housesigma.com/site/en/analyze/city-median-price-report)

HTML scraping to collect median pricing for detached houses, semi-detached houses, freehold townhouses, condo townhouses, and condo apartments in Canadian dollars.  Note that only **detached houses** are analysed in this study.

Data in this report are provided in cities and their communities, not in FSA postcodes.  However, these communities have naming close to neighbourhood from FSA postcode data source.  But some cleaning will still be needed, for example, the report of _Wexford-Maryvale_ community in  Scarborough (borough) is broken down into two neighbourhoods: _Wexford Heights_ and _Maryvale_ in FSA postcodes above.

### 2.5. Statistics Canada - 2016 Census

HTML scraping to find out demographic and economic data (such as average family income) in a FSA postcode area.

For example, hyperlink for M5H postcode (as Code1 parameter in the URL):
[https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1=M5H](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1=M5H)

## 3. Data Loading

In [1]:
# Import needed libraries

import io

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import numpy as np # library to handle data in a vectorized manner

import os # miscellaneous operating system interfaces

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import requests # library to handle URL requests

import time

# import k-means from clustering stage
from sklearn.cluster import KMeans

# html scraper
# !conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


In [2]:
# Column names used in dataframes in this notebook

COL_NAME_POSTCODE       = 'Postcode'
COL_NAME_BOROUGH        = 'Borough'
COL_NAME_NEIGHBOURHOOD  = 'Neighbourhood'
COL_NAME_LATITUDE       = 'Latitude'
COL_NAME_LONGITUDE      = 'Longitude'
COL_NAME_CLUSTER_LABELS = 'Cluster Labels'

COL_NAME_VENUE           = 'Venue' 
COL_NAME_VENUE_LATITUDE  = 'Venue Latitude'
COL_NAME_VENUE_LONGITUDE = 'Venue Longitude'
COL_NAME_VENUE_CATEGORY  = 'Venue Category'
COL_NAME_VENUE_FREQUENCY = 'Venue Frequency'

COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS = 'Avg Rating Elementary Schools'
COL_NAME_AVG_RATING_SECONDARY_SHCOOLS  = 'Avg Rating High Schools'

COL_NAME_SCHOOL_NAME        = 'School Name'
COL_NAME_SCHOOL_CITY        = 'School City'
COL_NAME_SCHOOL_RATING      = 'School Rating'
COL_NAME_SCHOOL_LEVEL       = 'School Level'
COL_NAME_SCHOOL_FULL_NAME   = 'School Full Name'
COL_NAME_SCHOOL_TYPE        = 'School Type'
COL_NAME_SCHOOL_ADDRESS     = 'School Address'
COL_NAME_SCHOOL_PROVINCE    = 'School Province'
COL_NAME_SCHOOL_POSTAL_CODE = 'School Postal Code'
COL_NAME_SCHOOL_PHONE       = 'School Phone'
COL_NAME_SCHOOL_DISTRICT    = 'School District'
COL_NAME_SCHOOL_LATITUDE    = 'School Latitude'
COL_NAME_SCHOOL_LONGITUDE   = 'School Longitude'

COL_NAME_TRANSPORTATION_COUNT = 'Transportation Count'

COL_NAME_HOUSESIGMA_COMMUNITY    = 'HouseSigma Community'
COL_NAME_HOUSESIGMA_MEDIAN_PRICE = 'HouseSigma Median Price'

COL_NAME_CENSUS_POPULATION_SIZE           = 'Census Population Size'
COL_NAME_CENSUS_MEDIAN_AGE                = 'Census Median Age'
COL_NAME_CENSUS_AVERAGE_HOUSEHOLD_SIZE    = 'Census Average Household Size'
COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN = 'Census Households With Children (%)'
COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS      = 'Census Number of Households'
COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME      = 'Census Median Family Income'
COL_NAME_CENSUS_IN_LABOUR_FORCE           = 'Census In Labour Force (%)'
COL_NAME_CENSUS_NON_IMMIGRANTS            = 'Census Non Immigrants (%)'
COL_NAME_CENSUS_IMMIGRANTS                = 'Census Immigrants (%)'
COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS   = 'Census Non Permanent Residents (%)'

# Special values

COL_VALUE_NOT_ASSIGNED = 'Not assigned'
COL_VALUE_TORONTO      = 'Toronto'
COL_VALUE_ELEMENTARY   = 'elementary'
COL_VALUE_SECONDARY    = 'secondary'

SECONDS_IN_A_DAY        = 24 * 3600
REFETCH_CACHE_OFTEN     = 14         # Days before refetching external data source to save locally
REFETCH_CACHE_QUARTERLY = 120        # Non-frequent changes data sources don't need to refetch frequently
WALKING_DISTANCE        = 800        # 10 min. walking distance
SCHOOL_DISTANCE         = 3000       # 3km
NUM_TOP_VENUES          = 5          # Number of top venues in each postcode area
MAX_SCHOOL_RATING       = 10.0
K_CLUSTERS              = 7          # Number of clusters for K-Means clustering

# Foursquare app settings

FOURSQUARE_SUBFOLDER         = './foursquare'
FOURSQUARE_APP_CLIENT_ID     = 'CLIENT_ID'
FOURSQUARE_APP_CLIENT_SECRET = 'CLIENT_SECRET'
FOURSQUARE_APP_VERSION       = 'VERSION'

# Foursquare category ids

FOURSQUARE_CATID_ELEMENTARY_SCHOOL      = '4f4533804b9074f6e4fb0105'
FOURSQUARE_CATID_HIGH_SCHOOL            = '4bf58dd8d48988d13d941735'
FOURSQUARE_CATID_MIDDLE_SCHOOL          = '4f4533814b9074f6e4fb0106'
FOURSQUARE_CATID_PRIVATE_SCHOOL         = '52e81612bcbc57f1066b7a46'

FOURSQUARE_CATID_BUS_STATION            = '4bf58dd8d48988d1fe931735'
FOURSQUARE_CATID_BUS_STOP               = '52f2ab2ebcbc57f1066b8b4f'
FOURSQUARE_CATID_TRAIN_STATION          = '4bf58dd8d48988d129951735'
FOURSQUARE_CATID_TRANSPORTATION_SERVICE = '54541b70498ea6ccd0204bff'

# Fraser Institute

FRASER_SUBFOLDER              = './fraser_institute'
FRASER_RATINGS_URL_ELEMENTARY = 'http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx?schooltype=elementary'
FRASER_RATINGS_URL_SECONDARY  = 'http://ontario.compareschoolrankings.org/secondary/SchoolsByRankLocationName.aspx?schooltype=secondary'

# Statistic Canada

STATISTIC_CANADA_SUBFOLDER = './statistic_canada'

### 3.1. Load Neighbourhoods in Toronto with Geospatial Coordinates

In [3]:
# Get the wiki web page

toronto_postal_codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_postal_codes_html = requests.get(toronto_postal_codes_url).text
# print(toronto_postal_codes_html)

In [4]:
# Parse html tags to extract information later

toronto_postal_codes_soap = BeautifulSoup(toronto_postal_codes_html)
# print(toronto_postal_codes_soap.prettify())

In [5]:
# Find the table with list of postal codes

postal_codes = toronto_postal_codes_soap.find('table', class_='wikitable sortable')

In [6]:
# Parse table data and put them into DataFrame
#
# Notes:
# 1. Not assigned boroughs are ignored
# 2. A not assigned neigbourhood will have the same name as its borough

columns = [COL_NAME_POSTCODE, COL_NAME_BOROUGH, COL_NAME_NEIGHBOURHOOD]
neighbourhoods = pd.DataFrame(columns=columns)

for row in postal_codes.find_all('tr'):
    cells = [cell.text.strip() for cell in row.find_all('td')]
    
    if len(cells) == 3:
        postal_code = cells[0]
        borough = cells[1]
        neighbourhood = cells[2]
    
        if borough != COL_VALUE_NOT_ASSIGNED:
            # only known boroughs are considered
            neighbourhoods = neighbourhoods.append({
                    COL_NAME_POSTCODE: postal_code,
                    COL_NAME_BOROUGH: borough,
                    # not assigned neighbourhood will have the same name as its borough
                    COL_NAME_NEIGHBOURHOOD: borough if neighbourhood == COL_VALUE_NOT_ASSIGNED else neighbourhood 
                },
                ignore_index=True
            )

print(neighbourhoods.shape)
neighbourhoods

(211, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


In [7]:
# Group neighbourhoods (separated with commas) of the same postal code into one row

neighbourhoods_normalized = pd.DataFrame(columns=columns)

for group_by in neighbourhoods.groupby([COL_NAME_POSTCODE, COL_NAME_BOROUGH]):
    postal_code = group_by[0][0];

    borough = group_by[0][1];

    # sort all neighbourhoods by name before combining them into one string separated with commas
    neighbourhoods_by_borough = group_by[1].sort_values(COL_NAME_NEIGHBOURHOOD)
    neighbourhood = ', '.join(neighbourhoods_by_borough[COL_NAME_NEIGHBOURHOOD].values)

    neighbourhoods_normalized = neighbourhoods_normalized.append({
            COL_NAME_POSTCODE: postal_code,
            COL_NAME_BOROUGH: borough,
            COL_NAME_NEIGHBOURHOOD: neighbourhood 
        },
        ignore_index=True
    )

print(neighbourhoods_normalized.shape)
neighbourhoods_normalized.head(10)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
# We are going to use the reloaded latitudes and longitudes for Toronto's postal codes

GEOSPATIAL_COORDINATES_URL = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"

csv = requests.get(GEOSPATIAL_COORDINATES_URL).content
geospatial_coordinates = pd.read_csv(io.StringIO(csv.decode('utf-8')))

geospatial_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
toronto_neighbourhoods = neighbourhoods_normalized

# merge geo coordinates with neighbourhood data to add latitude/longitude for each neighborhood
toronto_neighbourhoods = toronto_neighbourhoods.join(
    geospatial_coordinates.set_index('Postal Code'), on=COL_NAME_POSTCODE
)

print(toronto_neighbourhoods.shape)
toronto_neighbourhoods

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


#### Create a map of Toronto with neighbourhoods by postal codes superimposed on top.

In [10]:
# We will use Davisville North as center of the map;
# otherwise Lake Ontario will take up a lot of space in the bottom of the map

toronto_map_center = toronto_neighbourhoods[toronto_neighbourhoods[COL_NAME_NEIGHBOURHOOD] == 'Davisville North']

print('The geograpical coordinate of center of Toronto map are {}, {}.'.format(
    toronto_map_center[COL_NAME_LATITUDE].values[0],
    toronto_map_center[COL_NAME_LONGITUDE].values[0]
))

The geograpical coordinate of center of Toronto map are 43.7127511, -79.3901975.


In [11]:
# Create a map of Toronto with postal code locations superimposed on top

toronto_map = folium.Map(
    location=[toronto_map_center[COL_NAME_LATITUDE].values[0], toronto_map_center[COL_NAME_LONGITUDE].values[0]],
    zoom_start=11
)

# add markers with neighbourhood names as popup labels to the map
for lat, lng, borough, neighbourhood in zip(
    toronto_neighbourhoods[COL_NAME_LATITUDE],
    toronto_neighbourhoods[COL_NAME_LONGITUDE],
    toronto_neighbourhoods[COL_NAME_BOROUGH],
    toronto_neighbourhoods[COL_NAME_NEIGHBOURHOOD]):
    
    label = '{} : {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### 3.2. Load Venues in Each City of Toronto's Postcode

Using Foursquare API to find out top venues in each City of Toronto's postcode

In [12]:
# Load Foursquare CLIENT_ID and CLIENT_SECRET for API calls below

with open('{}/foursquare_app_settings.json'.format(FOURSQUARE_SUBFOLDER)) as json_file:
    foursquare_app_settings = json.load(json_file)

#### Let's create a function to find out venues to all the neighbourhoods in City of Toronto

In [13]:
def getNearbyVenues(foursquare_app_settings, postcodes, latitudes, longitudes, radius=WALKING_DISTANCE, venue_limit=100):
    # define the dataframe columns
    column_names = [
        COL_NAME_POSTCODE, 
        COL_NAME_LATITUDE, 
        COL_NAME_LONGITUDE, 
        COL_NAME_VENUE, 
        COL_NAME_VENUE_LATITUDE, 
        COL_NAME_VENUE_LONGITUDE, 
        COL_NAME_VENUE_CATEGORY
    ]

    # instantiate the dataframe
    nearby_venues = pd.DataFrame(columns=column_names)
    
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        # print(postcode)
        
        # check whether we have downloaded the venues for the postcode in last REFETCH_CACHE_OFTEN days
        # if not, we will request the venues from Foursqaure
        # this is to speed up of rerun of this notebook
        postcode_venues_filename = '{}/{}_venues.json'.format(FOURSQUARE_SUBFOLDER, postcode)
        if (
            os.path.exists(postcode_venues_filename)
            and ((time.time() - os.path.getctime(postcode_venues_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_OFTEN)
           ):
            # load venues from previous downloaded file
            with open(postcode_venues_filename) as json_file:
                postcode_veneues_json = json.load(json_file)
        else:
            # we need to fetch the venues for the given postcode from Foursquare
            
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                foursquare_app_settings[FOURSQUARE_APP_CLIENT_ID], 
                foursquare_app_settings[FOURSQUARE_APP_CLIENT_SECRET], 
                foursquare_app_settings[FOURSQUARE_APP_VERSION], 
                lat, 
                lng, 
                radius, 
                venue_limit
            )
            
            # make the GET request
            postcode_veneues_json = requests.get(url).json()
            
            # save the fetched venues into local file for future quick run
            with open(postcode_venues_filename, 'w') as json_file:
                    json.dump(postcode_veneues_json, json_file)

        # parse response
        results = postcode_veneues_json["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            nearby_venues = nearby_venues.append({
                    COL_NAME_POSTCODE: postcode,
                    COL_NAME_LATITUDE: lat,
                    COL_NAME_LONGITUDE: lng,
                    COL_NAME_VENUE: v['venue']['name'],
                    COL_NAME_VENUE_LATITUDE: v['venue']['location']['lat'],
                    COL_NAME_VENUE_LONGITUDE: v['venue']['location']['lng'],
                    COL_NAME_VENUE_CATEGORY: v['venue']['categories'][0]['name']
                },
                ignore_index=True
            )
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(
    foursquare_app_settings,
    postcodes=toronto_neighbourhoods[COL_NAME_POSTCODE],
    latitudes=toronto_neighbourhoods[COL_NAME_LATITUDE],
    longitudes=toronto_neighbourhoods[COL_NAME_LONGITUDE]
)

print(toronto_venues.shape)
toronto_venues

(3945, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,M1B,43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
4,M1B,43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
...,...,...,...,...,...,...,...
3940,M9V,43.739416,-79.588437,Canadian Tire,43.741844,-79.582294,Hardware Store
3941,M9V,43.739416,-79.588437,Pizza Pizza,43.733202,-79.592538,Pizza Place
3942,M9V,43.739416,-79.588437,Tim Hortons,43.742015,-79.589690,Coffee Shop
3943,M9W,43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location


#### Let's check how many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby(COL_NAME_POSTCODE)[COL_NAME_VENUE].count()

Postcode
M1B    12
M1C     5
M1E    15
M1G     6
M1H    22
       ..
M9N     6
M9P    10
M9R    14
M9V    13
M9W     2
Name: Venue, Length: 102, dtype: int64

#### Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues[COL_NAME_VENUE_CATEGORY].unique())))

# Top venue categories for clustering later
toronto_top_venue_categories = toronto_venues[COL_NAME_VENUE_CATEGORY].value_counts()[:(2 * NUM_TOP_VENUES)]

print('--- Top venue categories ---')
toronto_top_venue_categories

There are 327 uniques categories.
--- Top venue categories ---


Coffee Shop           303
Café                  169
Park                  117
Pizza Place           113
Restaurant            102
Italian Restaurant     97
Bakery                 94
Sandwich Place         78
Bar                    68
Hotel                  68
Name: Venue Category, dtype: int64

### 3.3. Load School Ratings in Each City of Toronto's Postcode

We will use Foursquare to find out close by schools in each postcode.  Then we will find the school ratings from Frase Institute web site.

Note that we do not take into school boundaries into considerations.  So the actual results would be different but our overall school rating averages should be close enough for our analysis.

#### Load School Ratings from Fraser Institute

In [17]:
def getSchoolDetails(school_name, city, fraser_institute_ratings_url, school_path):
    # print(school_name)

    # check whether we have downloaded the school details in last REFETCH_CACHE_QUARTERLY days
    # (school details don't change often if ever)
    # if not, we will request the school details from the given url and path
    # this is to speed up of rerun of this notebook
    school_details_filename = '{}/{}_{}_school_details.json'.format(
        FRASER_SUBFOLDER,
        school_name.replace('/', ' - '),
        city
    )

    if (
        os.path.exists(school_details_filename)
        and ((time.time() - os.path.getctime(school_details_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_QUARTERLY)
       ):
        # load venues from previous downloaded file
        with open(school_details_filename) as json_file:
            school_details = json.load(json_file)
    else:
        # we need to fetch the venues for the given postcode from Foursquare

        # Calc the absolute url from the rating url and relative path to school details web page
        url_paths = FRASER_RATINGS_URL_SECONDARY.split('/')
        school_url = '{}//{}{}'.format(url_paths[0], url_paths[2], school_path)

        # Get the school details
        school_html = requests.get(school_url).text

        # Parse html tags to extract information later
        school_soap = BeautifulSoup(school_html)

        # Find school info
        school_info = school_soap.find('span', id='ctl00_ContentPlaceHolder1_SchoolInfoDisplay')

        # Break downs the school info text limited by <br/> tags into individual details
        # <br/> tag has content.string == None
        school_info_details = [content for content in school_info.contents if content.string]

        school_full_name = school_info_details[0].string.strip()
        school_type = school_info_details[1].string.strip()
        school_address = school_info_details[2].string.strip()
        school_phone = school_info_details[4].string.replace('Phone Number: ', '').strip()
        school_district = school_info_details[5].string.replace('School District: ', '').strip()

        # Example of 4th (index 3) line: 'East Garafraxa, ON L9W 7J1'
        city_province_postal_code = school_info_details[3].string.split(',')
        school_city = city_province_postal_code[0]
        
        province_postal_code = city_province_postal_code[1].strip().split(' ')
        school_province = province_postal_code[0]
        
        if len(province_postal_code) == 2:
            # Add ' ' in the middle to get 'A1A 1A1' postal code format
            school_postal_code = province_postal_code[1][:3] + ' ' + province_postal_code[1][3:]
        elif len(province_postal_code) == 3:
            # Concatenate ['A1A', '1A1'] into 'A1A 1A1' postal code format
            school_postal_code = ' '.join(province_postal_code[1:])
        else:
            school_postal_code = ''

        school_details = {
            COL_NAME_SCHOOL_FULL_NAME: school_full_name,
            COL_NAME_SCHOOL_TYPE: school_type,
            COL_NAME_SCHOOL_ADDRESS: school_address,
            COL_NAME_SCHOOL_CITY: school_city,
            COL_NAME_SCHOOL_PROVINCE: school_province,
            COL_NAME_SCHOOL_POSTAL_CODE: school_postal_code,
            COL_NAME_SCHOOL_PHONE: school_phone,
            COL_NAME_SCHOOL_DISTRICT: school_district
        }
        
        # save the fetched school details into local file for future quick run
        with open(school_details_filename, 'w') as json_file:
                json.dump(school_details, json_file)

    if city != school_details[COL_NAME_SCHOOL_CITY]:
        print('WARN: Given city "{}" does not match the parsed value "{}" for school "{}"'.format(
            city,
            school_details[COL_NAME_SCHOOL_CITY],
            school_details[COL_NAME_SCHOOL_FULL_NAME]
        ))

    return school_details

In [18]:
def getSchoolRatings(fraser_institute_ratings_url, only_city = ''):
    school_level = fraser_institute_ratings_url.split('=')[-1]
    
    # check whether we have downloaded the school ratings in last REFETCH_CACHE_OFTEN days
    # if not, we will request the ratings from the given url
    # this is to speed up of rerun of this notebook
    school_ratings_filename = '{}/{}_{}_ratings.json'.format(FRASER_SUBFOLDER, school_level, only_city)
    if (
        os.path.exists(school_ratings_filename)
        and ((time.time() - os.path.getctime(school_ratings_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_OFTEN)
       ):
        # load venues from previous downloaded file
        school_ratings = pd.read_json(school_ratings_filename)
    else:
        # we need to fetch the venues for the given postcode from Foursquare

        # Get the school ratings web page
        school_ratings_html = requests.get(fraser_institute_ratings_url).text

        # Parse html tags to extract information later
        school_ratings_soap = BeautifulSoup(school_ratings_html)

        # Find the table with school ratings
        ratings_table = school_ratings_soap.find('table', class_='rating').find('table')

        # Parse table data and put them into DataFrame
        #
        # Notes:
        # 1. We only use 'School Name', 'City', and 'Rating' columns
        # 2. We only concern schools in Toronto city; ignoring schools from other cities
        # 3. Remove any '.' characters in the school name to avoid mismatch by spelling
        columns = [
            COL_NAME_SCHOOL_NAME,
            COL_NAME_SCHOOL_CITY,
            COL_NAME_SCHOOL_RATING,
            COL_NAME_SCHOOL_LEVEL,
            COL_NAME_SCHOOL_FULL_NAME,
            COL_NAME_SCHOOL_TYPE,
            COL_NAME_SCHOOL_ADDRESS,
            COL_NAME_SCHOOL_PROVINCE,
            COL_NAME_SCHOOL_POSTAL_CODE,
            COL_NAME_SCHOOL_PHONE,
            COL_NAME_SCHOOL_DISTRICT
        ]
        school_ratings = pd.DataFrame(columns=columns)

        for row in ratings_table.find_all('tr'):
            cells = [cell.text.strip() for cell in row.find_all('td', class_='tdcell')]

            # Note that the last (8th) column is of class='tdcell_no', so we have 7 columns
            if len(cells) == 7:
                school_name = cells[3]
                city = cells[4]
                rating = float(cells[5])
                
                # Ignore schools from other cities if only_city is given
                if not only_city or (only_city == city):
                    school_details_path = row.find_all('td', class_='tdcell')[3].find('a')['href']
                    school_details = getSchoolDetails(school_name, city, fraser_institute_ratings_url, school_details_path)

                    school_ratings = school_ratings.append({
                            COL_NAME_SCHOOL_NAME: school_name,
                            COL_NAME_SCHOOL_CITY: city,
                            COL_NAME_SCHOOL_RATING: float(rating),
                            COL_NAME_SCHOOL_LEVEL: school_level,
                            COL_NAME_SCHOOL_FULL_NAME: school_details[COL_NAME_SCHOOL_FULL_NAME],
                            COL_NAME_SCHOOL_TYPE: school_details[COL_NAME_SCHOOL_TYPE],
                            COL_NAME_SCHOOL_ADDRESS: school_details[COL_NAME_SCHOOL_ADDRESS],
                            COL_NAME_SCHOOL_PROVINCE: school_details[COL_NAME_SCHOOL_PROVINCE],
                            COL_NAME_SCHOOL_POSTAL_CODE: school_details[COL_NAME_SCHOOL_POSTAL_CODE],
                            COL_NAME_SCHOOL_PHONE: school_details[COL_NAME_SCHOOL_PHONE],
                            COL_NAME_SCHOOL_DISTRICT: school_details[COL_NAME_SCHOOL_DISTRICT]
                        },
                        ignore_index=True
                    )

        # save the fetched school ratings into local file for future quick run
        school_ratings.to_json(school_ratings_filename)

    return school_ratings

In [19]:
# Get elementary schools ratings in Toronto
elementary_school_ratings = getSchoolRatings(FRASER_RATINGS_URL_ELEMENTARY, 'Toronto')
print(elementary_school_ratings.shape)

(442, 11)


In [20]:
# Get secondary schools ratings in Toronto
secondary_school_ratings = getSchoolRatings(FRASER_RATINGS_URL_SECONDARY, 'Toronto')
print(secondary_school_ratings.shape)

(107, 11)


In [21]:
# Combine elementary and secondary school ratings into one table for rating search later
fraser_school_ratings = pd.concat([elementary_school_ratings, secondary_school_ratings], ignore_index=True)
fraser_school_ratings.sort_values(COL_NAME_SCHOOL_RATING, ascending=False, inplace=True)
fraser_school_ratings.reset_index(drop=True, inplace=True)

print(fraser_school_ratings.shape)
fraser_school_ratings

(549, 11)


,School Name,School City,School Rating,School Level,School Full Name,School Type,School Address,School Province,School Postal Code,School Phone,School District
0,Avondale Alternative,Toronto,10.0,elementary,Avondale Alternative Elementary School,Public,171 Avondale Ave,ON,M2N 2V4,416-395-3130,Toronto DSB
1,Havergal,Toronto,10.0,secondary,Havergal College,Private,1451 Avenue Rd,ON,M5N 2H9,416-483-3519,TO Private Inspected
2,Islamic Institute of Toronto,Toronto,10.0,elementary,Islamic Institute of Toronto Academy (IIT Acad...,Private,1630 NEILSON RD,ON,M1X 1S3,416-335-9173,TO Private Non-Inspected
3,Northmount,Toronto,10.0,elementary,Northmount School,Private,26 Mallard Rd,ON,M3B 1S3,416-449-8823,TO Private Non-Inspected
4,Sathya Sai,Toronto,10.0,elementary,Sathya Sai School of Toronto,Private,505 Ellesmere Rd 4,ON,M1R 4E5,416-297-7970,TO Private Non-Inspected
...,...,...,...,...,...,...,...,...,...,...,...
544,Oakwood,Toronto,1.5,secondary,Oakwood Collegiate Institute,Public,991 St Clair Ave W,ON,M6E 1A3,416-393-1780,Toronto DSB
545,Kipling,Toronto,1.4,secondary,Kipling Collegiate Institute,Public,380 The Westway,ON,M9R 1H4,416-394-7930,Toronto DSB
546,F H Miller,Toronto,0.7,elementary,F H Miller Junior Public School,Public,300 Caledonia Rd,ON,M6E 4T5,416-394-2336,Toronto DSB
547,Central,Toronto,0.2,secondary,Central Technical School,Public,725 Bathurst St,ON,M5S 2R5,416-393-0060,Toronto DSB


### Load and calculate school rating averages for each postcode

In [22]:
# Global variable to keep track of not found schools so we don't need to find it multiple times
not_found_schools = []

def getAvgSchoolRatingForPostcode(
    foursquare_app_settings,
    fraser_school_ratings,
    postcode,
    latitutde,
    longitude,
    school_level,
    category_ids, # A comma separated list of Foursquare school categories
    radius=SCHOOL_DISTANCE,
    school_limit=20):
    
    # check whether we have downloaded the schools for the postcode in last REFETCH_CACHE_QUARTERLY days
    # (schools in a postcode are don't change often if ever)
    # if not, we will request the schools from Foursqaure
    # this is to speed up of rerun of this notebook
    postcode_schools_filename = '{}/{}_schools_{}.json'.format(FOURSQUARE_SUBFOLDER, postcode, school_level)
    if (
        os.path.exists(postcode_schools_filename)
        and ((time.time() - os.path.getctime(postcode_schools_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_QUARTERLY)
       ):
        # load venues from previous downloaded file
        with open(postcode_schools_filename) as json_file:
            postcode_schools_json = json.load(json_file)
    else:
        # we need to fetch the venues for the given postcode from Foursquare

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            foursquare_app_settings[FOURSQUARE_APP_CLIENT_ID], 
            foursquare_app_settings[FOURSQUARE_APP_CLIENT_SECRET], 
            foursquare_app_settings[FOURSQUARE_APP_VERSION], 
            latitutde, 
            longitude, 
            radius, 
            school_limit,
            category_ids
        )

        # make the GET request
        postcode_schools_json = requests.get(url).json()

        # save the fetched venues into local file for future quick run
        with open(postcode_schools_filename, 'w') as json_file:
                json.dump(postcode_schools_json, json_file)

    # parse response
    if not('groups' in postcode_schools_json["response"]):
        # no school was found
        return 0.0

    results = postcode_schools_json["response"]['groups'][0]['items']

    # look for school details in the Foursquare response
    school_match_count = 0
    school_rating_total = 0
    for v in results:
        school_rating = -1 # not found
        
        school_full_name = v['venue']['name']
        # print(school_full_name)
        
        if school_full_name in not_found_schools:
            # We couldn't find the school before, no need to try again
            continue
        
        # find school by school name (without endings such as 'Public School')

        school_name = school_full_name
        # St. --> St (no period)
        school_name = school_name.replace('St.', 'St')
        # Correct known mis-spelling from Foursquare
        school_name = school_name.replace('Mary Thompson', 'Mary Thomson')
        school_name = school_name.replace('Runnymede Junior and Senior Public School', 'Runnymede')
        school_name = school_name.replace('Thistletown Collegiate Institute (T.C.I.)', 'Thistletown Collegiate Institute')
        # Remove common endings
        school_name = school_name.replace(' C I', '')
        school_name = school_name.replace(' C.I', '')
        school_name = school_name.replace(' C.I.', '')
        school_name = school_name.replace(' CI', '')
        school_name = school_name.replace(' Collegiate Institute', '')
        school_name = school_name.replace(' College School', '')
        school_name = school_name.replace(' Catholic', '')
        school_name = school_name.replace(' CSS', '')
        school_name = school_name.replace(' CS', '')
        school_name = school_name.replace(' HS', '')
        school_name = school_name.replace(' Junior High', '')
        school_name = school_name.replace(' Junior', '')
        school_name = school_name.replace(' Senior', '')
        school_name = school_name.replace(' JHS', '')
        school_name = school_name.replace(' Jr HS', '')
        school_name = school_name.replace(' JMS', '')
        school_name = school_name.replace(' Jr P S', '')
        school_name = school_name.replace(' Sr P S', '')
        school_name = school_name.replace(' P S', '')
        school_name = school_name.replace(' P.S', '')
        school_name = school_name.replace(' P.S.', '')
        school_name = school_name.replace(' PS', '')
        school_name = school_name.replace(' SS', '')
        school_name = school_name.replace(' Jr Public School', '')
        school_name = school_name.replace(' Sr Public School', '')
        school_name = school_name.replace(' Public School', '')
        school_name = school_name.replace(' public school', '')
        school_name = school_name.replace(' Elementary School', '')
        school_name = school_name.replace(' High School', '')
        school_name = school_name.replace(' high school', '')
        school_name = school_name.replace(' Highschool', '')
        school_name = school_name.replace(' Secondary School', '')
        school_name = school_name.replace(' Separate School', '')
        school_name = school_name.replace(' Elementary & Middle School', '')
        school_name = school_name.replace(' Middle School', '')
        school_name = school_name.replace(' Elementary', '')
        school_name = school_name.replace(' Schools', '')
        school_name = school_name.replace(' School', '')
        school_name = school_name.replace('. ', ' ')
        school_name = school_name.replace('.', ' ')
        # Case insensitve
        school_name = school_name.lower();

        # TODO: French school names won't match, like 'École Élémentaire Félix-Leclerc',
        #       because Foursquare's school names don't have French special characters
        #       Fraser's school names are correct
        for index, school in fraser_school_ratings.iterrows():
            if school[COL_NAME_SCHOOL_FULL_NAME].lower().find(school_name) >= 0:
                school_rating = school[COL_NAME_SCHOOL_RATING]
                break

        if (school_rating < 0) and ('postalCode' in v['venue']['location']):
            # find school by postal code if we could not match by school name
            postalCode = v['venue']['location']['postalCode']
            schools_by_postal_code = fraser_school_ratings[fraser_school_ratings[COL_NAME_SCHOOL_POSTAL_CODE] == postalCode]
            # skip if we find more than one school at the same postal code and we can't match by school name above
            if len(schools_by_postal_code) == 1:
                school_rating = schools_by_postal_code[COL_NAME_SCHOOL_RATING].iloc[0]

        if school_rating >= 0:
            school_match_count += 1
            school_rating_total += school_rating
            # print('{}: {}'.format(school_full_name, school_rating))
        else:
            # print('INFO: Cannot find rating for school: {} ==> {}'.format(school_full_name, school_name))
            not_found_schools.append(school_full_name)
    # end of for v in results
            
    if school_match_count == 0:
        return 0.0
    else:
        return (school_rating_total / school_match_count)

In [23]:
def getAvgSchoolRatings(
    foursquare_app_settings,
    school_ratings,
    postcodes,
    latitudes,
    longitudes,
    radius=SCHOOL_DISTANCE,
    school_limit=20):
    # define the dataframe columns
    column_names = [
        COL_NAME_POSTCODE,
        COL_NAME_LATITUDE,
        COL_NAME_LONGITUDE,
        COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS,
        COL_NAME_AVG_RATING_SECONDARY_SHCOOLS
    ]

    # instantiate the dataframe
    ratings = pd.DataFrame(columns=column_names)
    
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        # print(postcode)
        
        avg_rating_elementary_schools = getAvgSchoolRatingForPostcode(
            foursquare_app_settings,
            school_ratings,
            postcode,
            lat,
            lng,
            COL_VALUE_ELEMENTARY,
            FOURSQUARE_CATID_ELEMENTARY_SCHOOL
        )

        
        avg_rating_secondary_schools = getAvgSchoolRatingForPostcode(
            foursquare_app_settings,
            school_ratings,
            postcode,
            lat,
            lng,
            COL_VALUE_SECONDARY,
            ','.join([FOURSQUARE_CATID_HIGH_SCHOOL, FOURSQUARE_CATID_MIDDLE_SCHOOL])
        )
        
        # We ignore private schools for now because we don't know schol level they are

        ratings = ratings.append({
                COL_NAME_POSTCODE: postcode,
                COL_NAME_LATITUDE: lat,
                COL_NAME_LONGITUDE: lng,
                COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS: avg_rating_elementary_schools,
                COL_NAME_AVG_RATING_SECONDARY_SHCOOLS: avg_rating_secondary_schools
            },
            ignore_index=True
        )
    
    return(ratings)

In [24]:
toronto_avg_school_ratings = getAvgSchoolRatings(
    foursquare_app_settings,
    fraser_school_ratings,
    toronto_neighbourhoods[COL_NAME_POSTCODE],
    toronto_neighbourhoods[COL_NAME_LATITUDE],
    toronto_neighbourhoods[COL_NAME_LONGITUDE]
)

print(toronto_avg_school_ratings.shape)
toronto_avg_school_ratings

(103, 5)


,Postcode,Latitude,Longitude,Avg Rating Elementary Schools,Avg Rating High Schools
0,M1B,43.806686,-79.194353,6.240000,5.933333
1,M1C,43.784535,-79.160497,6.950000,4.125000
2,M1E,43.763573,-79.188711,5.033333,5.033333
3,M1G,43.770992,-79.216917,4.075000,5.750000
4,M1H,43.773136,-79.239476,5.200000,3.800000
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,6.500000,0.000000
99,M9P,43.696319,-79.532242,5.900000,7.800000
100,M9R,43.688905,-79.554724,5.900000,7.266667
101,M9V,43.739416,-79.588437,5.200000,7.966667


In [25]:
fraser_school_ratings[fraser_school_ratings[COL_NAME_SCHOOL_POSTAL_CODE] == 'M1K 3G2'][COL_NAME_SCHOOL_RATING].iloc[0]

4.0

### 3.4. Load Transportation in Each City of Toronto's Postcode

We will use Foursquare to find out how many transportation (like bus stops) in each postcode.

Note that we don't know take into account bus or train service lines; nor how convenient for a person in a postcode going to another postcode area far away in City of Toronto.

#### Let's create a function to find out transportation points to all the neighbourhoods in City of Toronto

In [26]:
def getNearbyTransportaion(foursquare_app_settings, postcodes, latitudes, longitudes, radius=WALKING_DISTANCE, venue_limit=100):
    # define the dataframe columns
    column_names = [
        COL_NAME_POSTCODE, 
        COL_NAME_LATITUDE, 
        COL_NAME_LONGITUDE, 
        COL_NAME_TRANSPORTATION_COUNT
    ]

    # instantiate the dataframe
    nearby_transportation_points = pd.DataFrame(columns=column_names)
    
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        # print(postcode)
        
        # check whether we have downloaded the transportations for the postcode in last REFETCH_CACHE_QUARTERLY days
        # if not, we will request the transportations from Foursqaure
        # this is to speed up of rerun of this notebook
        postcode_tranportations_filename = '{}/{}_transportations.json'.format(FOURSQUARE_SUBFOLDER, postcode)
        if (
            os.path.exists(postcode_tranportations_filename)
            and ((time.time() - os.path.getctime(postcode_tranportations_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_QUARTERLY)
           ):
            # load venues from previous downloaded file
            with open(postcode_tranportations_filename) as json_file:
                postcode_transportations_json = json.load(json_file)
        else:
            # we need to fetch the venues for the given postcode from Foursquare
            
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
                foursquare_app_settings[FOURSQUARE_APP_CLIENT_ID], 
                foursquare_app_settings[FOURSQUARE_APP_CLIENT_SECRET], 
                foursquare_app_settings[FOURSQUARE_APP_VERSION], 
                lat, 
                lng, 
                radius, 
                venue_limit,
                ','.join([FOURSQUARE_CATID_BUS_STATION, FOURSQUARE_CATID_BUS_STOP, FOURSQUARE_CATID_TRAIN_STATION])
            )
            
            # make the GET request
            postcode_transportations_json = requests.get(url).json()
            
            # save the fetched venues into local file for future quick run
            with open(postcode_tranportations_filename, 'w') as json_file:
                    json.dump(postcode_transportations_json, json_file)

        # parse response
        if not('groups' in postcode_transportations_json["response"]):
            # no transportation was found
            transportation_points = 0
        else:
            results = postcode_transportations_json["response"]['groups'][0]['items']
            transportation_points = len(results)
        
        nearby_transportation_points = nearby_transportation_points.append({
                COL_NAME_POSTCODE: postcode,
                COL_NAME_LATITUDE: lat,
                COL_NAME_LONGITUDE: lng,
                COL_NAME_TRANSPORTATION_COUNT: transportation_points
            },
            ignore_index=True
        )

    nearby_transportation_points[COL_NAME_TRANSPORTATION_COUNT] = nearby_transportation_points[COL_NAME_TRANSPORTATION_COUNT].astype(int)

    return(nearby_transportation_points)

In [27]:
toronto_transportations = getNearbyTransportaion(
    foursquare_app_settings,
    toronto_neighbourhoods[COL_NAME_POSTCODE],
    toronto_neighbourhoods[COL_NAME_LATITUDE],
    toronto_neighbourhoods[COL_NAME_LONGITUDE]
)

print(toronto_transportations.shape)
toronto_transportations

(103, 4)


,Postcode,Latitude,Longitude,Transportation Count
0,M1B,43.806686,-79.194353,3
1,M1C,43.784535,-79.160497,1
2,M1E,43.763573,-79.188711,5
3,M1G,43.770992,-79.216917,0
4,M1H,43.773136,-79.239476,1
...,...,...,...,...
98,M9N,43.706876,-79.518188,0
99,M9P,43.696319,-79.532242,1
100,M9R,43.688905,-79.554724,7
101,M9V,43.739416,-79.588437,3


### 3.4. HouseSigma - Median Price Report

[https://housesigma.com/site/en/analyze/city-median-price-report](https://housesigma.com/site/en/analyze/city-median-price-report)

| House Type         | house_type parameter |
|:-------------------|:---------------------|
| Detached           | D.                   |
| Semi-Detached      | S.                   |
| Freehold Townhouse | A.                   |
| Condo Townhouse    | T.                   |
| Condo Apt          | C.                   |

After looking into Detached houses information, it is not easy to match communities to Toronto neighbourhoods.

We will not be able to use this data source.

### 3.5. Statistics Canada - 2016 Census

We will parse the HTML pages from Statistics Canada for the following data and HTML headers attribute value to be parsed to get the data's numeric values:

| Data                                | HTML headers attribute value |
|:------------------------------------|:-----------------------------|
| Population size                     | 'L1000 geo1 total1'          |
| Median age                          | 'L2033 geo1 total1'          |
| Average household size              | 'L3017 geo1 total1'          |
| Without children in a census family | 'L6003 geo1 total1'          |
| With children in a census family    | 'L6004 geo1 total1'          |
| Number of households                | 'L6001 geo1 total1'          |
| Median total income                 | 'L13001 geo1 total1'         |
| Non-immigrants                      | 'L18001 geo1 total1'         |
| ImmigrantsCensus                    | 'L18002 geo1 total1'         |
| Non-permanent residents             | 'L18010 geo1 total1'         |

#### Let's create a function to get the 2016 Census data for each postcode

In [28]:
# It is possible that the cell has invalid numeric value
# for example, when thd data is not available, the cell value may be 'x'
def getCensusValueAsfloat(census_soap, headers_label):
    try:
        tag = census_soap.find('td', headers=headers_label)

        # Removing ',' separate thousands (e.g 68,000.00)
        cell = tag.text.strip().replace(',', '')
        
        return(float(cell))
    except:
        # Return 0.00 if we cannot parse the census data value
        return 0.00

In [29]:
def getCensusData(postcodes):
    # define the dataframe columns
    column_names = [
        COL_NAME_POSTCODE,
        COL_NAME_CENSUS_POPULATION_SIZE,
        COL_NAME_CENSUS_MEDIAN_AGE,
        COL_NAME_CENSUS_AVERAGE_HOUSEHOLD_SIZE,
        COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN,
        COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS,
        COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME,
        COL_NAME_CENSUS_NON_IMMIGRANTS,
        COL_NAME_CENSUS_IMMIGRANTS,
        COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS
    ]

    # instantiate the dataframe
    census_data = pd.DataFrame(columns=column_names)
    
    for postcode in postcodes:
        # print(postcode)
        
        # check whether we have downloaded the census data for the postcode in last REFETCH_CACHE_QUARTERLY days
        # if not, we will request the census data from Statistic Canada
        # this is to speed up of rerun of this notebook
        postcode_census_filename = '{}/{}_census.html'.format(STATISTIC_CANADA_SUBFOLDER, postcode)
        if (
            os.path.exists(postcode_census_filename)
            and ((time.time() - os.path.getctime(postcode_census_filename)) // SECONDS_IN_A_DAY <= REFETCH_CACHE_QUARTERLY)
           ):
            # load venues from previous downloaded file
            with open(postcode_census_filename) as html_file:
                postcode_census_html = html_file.read()
        else:
            # we need to fetch the census data for the given postcode from Statistic Canada
            
            # create the API request URL
            url = 'https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1={}'.format(
                postcode
            )
            
            # make the GET request
            postcode_census_html = requests.get(url).text
            
            # save the fetched venues into local file for future quick run
            with open(postcode_census_filename, 'w') as html_file:
                    html_file.write(postcode_census_html)

        # Parse html tags to extract information later
        census_soap = BeautifulSoup(postcode_census_html)
        
        # Find census data
        population_size = int(getCensusValueAsfloat(census_soap, 'L1000 geo1 total1'))

        median_age = getCensusValueAsfloat(census_soap, 'L2033 geo1 total1')
        
        average_household_size = getCensusValueAsfloat(census_soap, 'L3017 geo1 total1')
        
        without_children_census_family = int(getCensusValueAsfloat(census_soap, 'L6003 geo1 total1'))
        with_children_census_family = int(getCensusValueAsfloat(census_soap, 'L6004 geo1 total1'))
        total_census_family = without_children_census_family + without_children_census_family
        if without_children_census_family == 0:
            # Avoid ZeroDivisionError below
            without_children_census_family = 1
        
        number_of_households = int(getCensusValueAsfloat(census_soap, 'L6001 geo1 total1'))
        
        median_total_income = getCensusValueAsfloat(census_soap, 'L13001 geo1 total1')
        
        non_immigrants = int(getCensusValueAsfloat(census_soap, 'L18001 geo1 total1'))
        immigrants = int(getCensusValueAsfloat(census_soap, 'L18002 geo1 total1'))
        non_permanent_residents = int(getCensusValueAsfloat(census_soap, 'L18010 geo1 total1'))
        total_immigrant_status = non_immigrants + immigrants + non_permanent_residents
        if total_immigrant_status == 0:
            # Avoid ZeroDivisionError below
            total_immigrant_status = 1
    
        census_data = census_data.append({
                COL_NAME_POSTCODE: postcode,
                COL_NAME_CENSUS_POPULATION_SIZE: population_size,
                COL_NAME_CENSUS_MEDIAN_AGE: median_age,
                COL_NAME_CENSUS_AVERAGE_HOUSEHOLD_SIZE: average_household_size,
                COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN: round(100.0 * with_children_census_family / (without_children_census_family + with_children_census_family), 2),
                COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS: number_of_households,
                COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME: median_total_income,
                COL_NAME_CENSUS_NON_IMMIGRANTS: round(100 * non_immigrants / total_immigrant_status, 2),
                COL_NAME_CENSUS_IMMIGRANTS: round(100 * immigrants / total_immigrant_status, 2),
                COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS: round(100 * non_permanent_residents / total_immigrant_status, 2)
            },
            ignore_index=True
        )
        
    census_data[COL_NAME_CENSUS_POPULATION_SIZE] = census_data[COL_NAME_CENSUS_POPULATION_SIZE].astype(int)
    census_data[COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS] = census_data[COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS].astype(int)
        
    return(census_data)

In [30]:
toronto_census = getCensusData(
    toronto_neighbourhoods[COL_NAME_POSTCODE]
)

print(toronto_census.shape)
toronto_census

(103, 10)


,Postcode,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
0,M1B,66108,38.2,3.3,80.10,20230,69126.0,38.21,59.92,1.87
1,M1C,35626,44.0,3.1,70.14,11275,109785.0,54.12,44.89,0.99
2,M1E,46943,42.2,2.7,72.57,17160,62047.0,50.74,47.79,1.47
3,M1G,29690,37.2,3.0,76.57,9765,54450.0,40.54,56.03,3.43
4,M1H,24383,38.1,2.7,70.77,8985,58492.0,37.27,57.95,4.77
...,...,...,...,...,...,...,...,...,...,...
98,M9N,25074,39.0,2.4,73.68,10170,50545.0,49.56,47.74,2.71
99,M9P,20874,45.9,2.6,67.12,7905,73425.0,55.01,43.82,1.17
100,M9R,33743,40.6,2.7,70.89,12335,63032.0,47.34,50.69,1.97
101,M9V,55959,35.9,3.3,80.15,16805,59760.0,34.74,62.55,2.71


In [31]:
# Postcode areas which very sparse population and lack of census data
# These should be grouped into a cluster later on

toronto_census[toronto_census[COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS] <= 1000]

,Postcode,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
60,M5K,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
61,M5L,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
69,M5W,15,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
70,M5X,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
85,M7A,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
86,M7R,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
87,M7Y,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [32]:
# Postcode areas which have highest family income

toronto_census.sort_values(by=COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME, ascending=False)[:10]

,Postcode,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
44,M4N,15330,44.1,2.5,63.79,6190,137758.0,73.07,25.44,1.48
62,M5M,25975,41.2,2.8,71.06,9405,136982.0,72.78,25.73,1.49
90,M8X,10787,48.1,2.3,57.33,4525,119097.0,73.12,25.51,1.37
38,M4G,19076,43.3,2.4,67.70,7730,115833.0,76.04,22.71,1.25
20,M2L,11717,45.0,2.9,69.69,3975,115493.0,47.49,49.94,2.58
1,M1C,35626,44.0,3.1,70.14,11275,109785.0,54.12,44.89,0.99
26,M3B,13324,47.1,2.6,66.10,5000,108267.0,50.30,47.25,2.45
23,M2P,7843,45.8,2.6,66.04,3020,107520.0,51.35,46.47,2.19
50,M4W,14561,46.1,2.0,46.52,7265,101146.0,64.01,31.22,4.76
48,M4T,10463,45.0,2.0,50.19,5210,96571.0,73.45,24.00,2.55


## 4. Data Analysis

### 4.1. Analyze Venues in Each Postcode

#### Analyze Each Postal Code Area

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[[COL_NAME_VENUE_CATEGORY]], prefix="", prefix_sep="")

print(toronto_onehot.shape)
print(type(toronto_onehot))
toronto_onehot[toronto_top_venue_categories.index.tolist()]

(3945, 327)
<class 'pandas.core.frame.DataFrame'>


,Coffee Shop,Café,Park,Pizza Place,Restaurant,Italian Restaurant,Bakery,Sandwich Place,Bar,Hotel
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3940,0,0,0,0,0,0,0,0,0,0
3941,0,0,0,1,0,0,0,0,0,0
3942,1,0,0,0,0,0,0,0,0,0
3943,0,0,0,0,0,0,0,0,0,0


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[[COL_NAME_VENUE_CATEGORY]], prefix="", prefix_sep="")

# we limit to only the top venue categories to avoid clustering bias
# when we add other neighbourhoods' data due to high number of venue categories
toronto_onehot = toronto_onehot[toronto_top_venue_categories.index.tolist()]

# add Postcode column back to dataframe
toronto_onehot[COL_NAME_POSTCODE] = toronto_venues[COL_NAME_POSTCODE]

# move Postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot

(3945, 11)


,Postcode,Coffee Shop,Café,Park,Pizza Place,Restaurant,Italian Restaurant,Bakery,Sandwich Place,Bar,Hotel
0,M1B,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,0
2,M1B,0,0,0,0,0,0,0,0,0,0
3,M1B,0,0,0,0,0,0,0,0,0,0
4,M1B,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3940,M9V,0,0,0,0,0,0,0,0,0,0
3941,M9V,0,0,0,1,0,0,0,0,0,0
3942,M9V,1,0,0,0,0,0,0,0,0,0
3943,M9W,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by postcode and by taking the mean of the frequency of occurrence of each category

In [35]:
toronto_grouped = toronto_onehot.groupby(COL_NAME_POSTCODE).mean().reset_index()

# Put empty row for postcode without venues from Foursquare
# We need them later when we include school, transportation, and census data
for postcode in pd.concat([toronto_neighbourhoods[COL_NAME_POSTCODE], toronto_grouped[COL_NAME_POSTCODE]]).drop_duplicates(keep=False):
    print('No venues: ', postcode)
    toronto_grouped = toronto_grouped.append({
            COL_NAME_POSTCODE: postcode
        },
        ignore_index=True
    )

# Order by postcode in case we append postcode which does not have venues
toronto_grouped.fillna(0, inplace=True)
toronto_grouped.sort_values(by=COL_NAME_POSTCODE, inplace=True)
toronto_grouped.reset_index(drop=True, inplace=True)

print(toronto_grouped.shape)
toronto_grouped

No venues:  M1X
(103, 11)


,Postcode,Coffee Shop,Café,Park,Pizza Place,Restaurant,Italian Restaurant,Bakery,Sandwich Place,Bar,Hotel
0,M1B,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,M1C,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.000000,0.000000,0.2,0.0
2,M1E,0.133333,0.0,0.000000,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,M1G,0.333333,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4,M1H,0.090909,0.0,0.000000,0.000000,0.0,0.0,0.090909,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,0.000000,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.000000,0.0,0.0
99,M9P,0.100000,0.0,0.000000,0.100000,0.0,0.0,0.000000,0.100000,0.0,0.0
100,M9R,0.071429,0.0,0.000000,0.071429,0.0,0.0,0.000000,0.071429,0.0,0.0
101,M9V,0.076923,0.0,0.000000,0.230769,0.0,0.0,0.000000,0.076923,0.0,0.0


#### Let's print each postcode along with the most common venues

In [36]:
for hood in toronto_grouped[COL_NAME_POSTCODE]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped[COL_NAME_POSTCODE] == hood].T.reset_index()
    temp.columns = [COL_NAME_VENUE, COL_NAME_VENUE_FREQUENCY]
    temp = temp.iloc[1:] # remove the row of Neighbourhood (Neighborhood column before transpose operation)
    temp[COL_NAME_VENUE_FREQUENCY] = temp[COL_NAME_VENUE_FREQUENCY].astype(float)
    temp = temp.round({COL_NAME_VENUE_FREQUENCY: 2})
    print(
        temp.sort_values(COL_NAME_VENUE_FREQUENCY, ascending=False)
            .reset_index(drop=True)
            .head(NUM_TOP_VENUES)
    )
    print('\n')

----M1B----
         Venue  Venue Frequency
0  Coffee Shop             0.08
1         Café             0.00
2         Park             0.00
3  Pizza Place             0.00
4   Restaurant             0.00


----M1C----
                Venue  Venue Frequency
0  Italian Restaurant              0.2
1                 Bar              0.2
2         Coffee Shop              0.0
3                Café              0.0
4                Park              0.0


----M1E----
         Venue  Venue Frequency
0  Pizza Place             0.20
1  Coffee Shop             0.13
2         Café             0.00
3         Park             0.00
4   Restaurant             0.00


----M1G----
         Venue  Venue Frequency
0  Coffee Shop             0.33
1         Park             0.33
2         Café             0.00
3  Pizza Place             0.00
4   Restaurant             0.00


----M1H----
         Venue  Venue Frequency
0  Coffee Shop             0.09
1       Bakery             0.09
2         Café            

4   Restaurant             0.02


----M4K----
                Venue  Venue Frequency
0         Coffee Shop             0.07
1                Café             0.04
2                Park             0.04
3  Italian Restaurant             0.03
4         Pizza Place             0.02


----M4L----
                Venue  Venue Frequency
0         Coffee Shop             0.06
1                Café             0.04
2      Sandwich Place             0.04
3                Park             0.02
4  Italian Restaurant             0.02


----M4M----
         Venue  Venue Frequency
0       Bakery             0.07
1         Café             0.06
2  Coffee Shop             0.05
3          Bar             0.05
4         Park             0.03


----M4N----
         Venue  Venue Frequency
0  Coffee Shop             0.14
1         Café             0.14
2         Park             0.14
3   Restaurant             0.14
4  Pizza Place             0.00


----M4P----
                Venue  Venue Frequency
0      

4     Coffee Shop             0.00


----M6N----
            Venue  Venue Frequency
0            Park             0.12
1     Coffee Shop             0.08
2     Pizza Place             0.04
3  Sandwich Place             0.04
4             Bar             0.04


----M6P----
                Venue  Venue Frequency
0                Café             0.06
1  Italian Restaurant             0.06
2                 Bar             0.06
3         Coffee Shop             0.05
4                Park             0.05


----M6R----
         Venue  Venue Frequency
0          Bar             0.06
1  Coffee Shop             0.04
2  Pizza Place             0.04
3   Restaurant             0.04
4         Café             0.03


----M6S----
                Venue  Venue Frequency
0         Coffee Shop             0.11
1                Café             0.08
2         Pizza Place             0.05
3  Italian Restaurant             0.05
4              Bakery             0.05


----M7A----
                Venue  Ve

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [37]:
# Always return a list of NUM_TOP_VENUES but value will be '' (empty string)
# if its mean is close to zero
def returnTopVenues(row, num_top_venues = NUM_TOP_VENUES):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    topVenues = []
    for index in range(0, NUM_TOP_VENUES):
        if row_categories_sorted.iloc[index] >= 0.001:
            topVenues.append(row_categories_sorted.index.values[index])
        else:
            topVenues.append('')
    
    return(topVenues)

Now let's create the new dataframe and display the top venues for each neighborhood.

In [38]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = [COL_NAME_POSTCODE]
for ind in np.arange(NUM_TOP_VENUES):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted[COL_NAME_POSTCODE] = toronto_grouped[COL_NAME_POSTCODE]

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = returnTopVenues(toronto_grouped.iloc[ind, :])

print(toronto_venues_sorted.shape)
toronto_venues_sorted

(103, 6)


,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Coffee Shop,,,,
1,M1C,Bar,Italian Restaurant,,,
2,M1E,Pizza Place,Coffee Shop,,,
3,M1G,Park,Coffee Shop,,,
4,M1H,Bakery,Coffee Shop,,,
...,...,...,...,...,...,...
98,M9N,Pizza Place,,,,
99,M9P,Sandwich Place,Pizza Place,Coffee Shop,,
100,M9R,Sandwich Place,Pizza Place,Coffee Shop,,
101,M9V,Pizza Place,Sandwich Place,Coffee Shop,,


### 4.2. Consolidate with School Ratings

In [39]:
cols = [
    COL_NAME_POSTCODE,
    COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS,
    COL_NAME_AVG_RATING_SECONDARY_SHCOOLS
]

toronto_venues_school = pd.merge(
    toronto_grouped,
    toronto_avg_school_ratings[cols],
    on=COL_NAME_POSTCODE
)

toronto_venues_school_sorted = pd.merge(
    toronto_venues_sorted,
    toronto_avg_school_ratings[cols],
    on=COL_NAME_POSTCODE
)

### 4.3. Consolidate with Transportation Data

In [40]:
cols = [COL_NAME_POSTCODE, COL_NAME_TRANSPORTATION_COUNT]

toronto_venues_school_transportation = pd.merge(
    toronto_venues_school,
    toronto_transportations[cols],
    on=COL_NAME_POSTCODE
)

toronto_venues_school_transportation_sorted = pd.merge(
    toronto_venues_school_sorted,
    toronto_transportations[cols],
    on=COL_NAME_POSTCODE
)

### 4.4. Consolidate with Cesus Data

In [41]:
cols = [
    COL_NAME_POSTCODE,
    COL_NAME_CENSUS_POPULATION_SIZE,
    COL_NAME_CENSUS_MEDIAN_AGE,
    COL_NAME_CENSUS_AVERAGE_HOUSEHOLD_SIZE,
    COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN,
    COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS,
    COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME,
    COL_NAME_CENSUS_NON_IMMIGRANTS,
    COL_NAME_CENSUS_IMMIGRANTS,
    COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS
]

toronto_venues_school_transportation_census = pd.merge(
    toronto_venues_school_transportation,
    toronto_census[cols],
    on=COL_NAME_POSTCODE
)

toronto_venues_school_transportation_census_sorted = pd.merge(
    toronto_venues_school_transportation_sorted,
    toronto_census[cols],
    on=COL_NAME_POSTCODE
)

### 4.5. Normalize Consolidated Toronto Data

In [42]:
# Function to normalize give numeric columns of the dataframe
# noralized values will be always between 0 and 1
# Source: https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame/48651066

def normalize(df, columns):
    result = df.copy()
    for feature_name in columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [43]:
# Columns (not venues and not percentage) that have numeric values which be normalized between 0 and 1
cols_to_norm = [
    COL_NAME_TRANSPORTATION_COUNT,
    COL_NAME_CENSUS_POPULATION_SIZE,
    COL_NAME_CENSUS_MEDIAN_AGE,
    COL_NAME_CENSUS_AVERAGE_HOUSEHOLD_SIZE,
    COL_NAME_CENSUS_NUMBER_OF_HOUSEHOLDS,
    COL_NAME_CENSUS_MEDIAN_FAMILY_INCOME
]

toronto_clustering_data = normalize(toronto_venues_school_transportation_census, cols_to_norm)

# School ratings should be divided by MAX_SCHOOL_RATING
toronto_clustering_data[COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS] = toronto_clustering_data[COL_NAME_AVG_RATING_ELEMENTARY_SCHOOLS] / MAX_SCHOOL_RATING
toronto_clustering_data[COL_NAME_AVG_RATING_SECONDARY_SHCOOLS] = toronto_clustering_data[COL_NAME_AVG_RATING_SECONDARY_SHCOOLS] / MAX_SCHOOL_RATING

# For columns with percentage values, we will normalize betwee 0 and 1 (instead of 0 and 100)
toronto_clustering_data[COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN] = toronto_clustering_data[COL_NAME_CENSUS_HOUSEHOULDS_WITH_CHILDREN] / 100
toronto_clustering_data[COL_NAME_CENSUS_NON_IMMIGRANTS] = toronto_clustering_data[COL_NAME_CENSUS_NON_IMMIGRANTS] / 100
toronto_clustering_data[COL_NAME_CENSUS_IMMIGRANTS] = toronto_clustering_data[COL_NAME_CENSUS_IMMIGRANTS] / 100
toronto_clustering_data[COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS] = toronto_clustering_data[COL_NAME_CENSUS_NON_PERMANENT_RESIDENTS] / 100

print(toronto_clustering_data.shape)
toronto_clustering_data

(103, 23)


,Postcode,Coffee Shop,Café,Park,Pizza Place,Restaurant,Italian Restaurant,Bakery,Sandwich Place,Bar,Hotel,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
0,M1B,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.624000,0.593333,0.061224,0.871023,0.794179,0.804878,0.8010,0.599141,0.501793,0.3821,0.5992,0.0187
1,M1C,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.000000,0.000000,0.2,0.0,0.695000,0.412500,0.020408,0.469399,0.914761,0.756098,0.7014,0.333926,0.796941,0.5412,0.4489,0.0099
2,M1E,0.133333,0.0,0.000000,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.503333,0.503333,0.102041,0.618509,0.877339,0.658537,0.7257,0.508219,0.450406,0.5074,0.4779,0.0147
3,M1G,0.333333,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.407500,0.575000,0.000000,0.391188,0.773389,0.731707,0.7657,0.289205,0.395258,0.4054,0.5603,0.0343
4,M1H,0.090909,0.0,0.000000,0.000000,0.0,0.0,0.090909,0.000000,0.0,0.0,0.520000,0.380000,0.020408,0.321264,0.792100,0.658537,0.7077,0.266104,0.424600,0.3727,0.5795,0.0477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,0.000000,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.000000,0.0,0.0,0.650000,0.000000,0.000000,0.330369,0.810811,0.585366,0.7368,0.301199,0.366912,0.4956,0.4774,0.0271
99,M9P,0.100000,0.0,0.000000,0.100000,0.0,0.0,0.000000,0.100000,0.0,0.0,0.590000,0.780000,0.020408,0.275031,0.954262,0.634146,0.6712,0.234118,0.533000,0.5501,0.4382,0.0117
100,M9R,0.071429,0.0,0.000000,0.071429,0.0,0.0,0.000000,0.071429,0.0,0.0,0.590000,0.726667,0.142857,0.444589,0.844075,0.658537,0.7089,0.365319,0.457556,0.4734,0.5069,0.0197
101,M9V,0.076923,0.0,0.000000,0.230769,0.0,0.0,0.000000,0.076923,0.0,0.0,0.520000,0.796667,0.061224,0.737302,0.746362,0.804878,0.8015,0.497705,0.433804,0.3474,0.6255,0.0271


### 4.6. Cluster Postcodes

In [44]:
# We don't need postcode for K-Means training
toronto_clustering = toronto_clustering_data.drop(COL_NAME_POSTCODE, 1)

# run k-means clustering
kmeans = KMeans(n_clusters=K_CLUSTERS, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_.shape)
print(kmeans.labels_)

print('Cluster label counts:')
for i, count in enumerate(np.bincount(kmeans.labels_)):
    print('Cluster label {}: {}'.format(i, count))

(103,)
[6 4 6 6 1 6 6 6 1 4 6 1 6 6 6 6 1 0 6 0 4 0 3 4 6 6 4 6 6 1 1 1 1 6 1 4 6
 4 4 1 4 4 4 4 4 4 4 4 4 0 4 1 3 3 1 5 5 1 5 5 2 2 4 4 4 4 1 1 3 2 2 1 6 4
 6 4 3 4 3 1 6 6 3 4 4 2 2 2 3 4 4 4 4 0 4 4 0 0 1 4 6 6 6]
Cluster label counts:
Cluster label 0: 7
Cluster label 1: 18
Cluster label 2: 7
Cluster label 3: 8
Cluster label 4: 34
Cluster label 5: 4
Cluster label 6: 25


#### Let's create a new dataframe that includes the cluster as well as the top venues and other data for each postcode

In [45]:
# add clustering labels column
if COL_NAME_CLUSTER_LABELS in toronto_venues_school_transportation_census_sorted.columns:
    # We just update the cluster label column if the column has been inserted
    # like when we rerun this cell manually
    toronto_venues_school_transportation_census_sorted[COL_NAME_CLUSTER_LABELS] = kmeans.labels_
else:
    toronto_venues_school_transportation_census_sorted.insert(0, COL_NAME_CLUSTER_LABELS, kmeans.labels_)

toronto_merged = toronto_neighbourhoods

# merge all details with toronto_neigbourhoods to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(
    toronto_venues_school_transportation_census_sorted.set_index(COL_NAME_POSTCODE),
    on=COL_NAME_POSTCODE
)

print(toronto_merged.shape)
toronto_merged # check the last columns!

(103, 23)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,6,Coffee Shop,,,,,6.240000,5.933333,3,66108,38.2,3.3,80.10,20230,69126.0,38.21,59.92,1.87
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,4,Bar,Italian Restaurant,,,,6.950000,4.125000,1,35626,44.0,3.1,70.14,11275,109785.0,54.12,44.89,0.99
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,6,Pizza Place,Coffee Shop,,,,5.033333,5.033333,5,46943,42.2,2.7,72.57,17160,62047.0,50.74,47.79,1.47
3,M1G,Scarborough,Woburn,43.770992,-79.216917,6,Park,Coffee Shop,,,,4.075000,5.750000,0,29690,37.2,3.0,76.57,9765,54450.0,40.54,56.03,3.43
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Bakery,Coffee Shop,,,,5.200000,3.800000,1,24383,38.1,2.7,70.77,8985,58492.0,37.27,57.95,4.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,1,Pizza Place,,,,,6.500000,0.000000,0,25074,39.0,2.4,73.68,10170,50545.0,49.56,47.74,2.71
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,4,Sandwich Place,Pizza Place,Coffee Shop,,,5.900000,7.800000,1,20874,45.9,2.6,67.12,7905,73425.0,55.01,43.82,1.17
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,6,Sandwich Place,Pizza Place,Coffee Shop,,,5.900000,7.266667,7,33743,40.6,2.7,70.89,12335,63032.0,47.34,50.69,1.97
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,6,Pizza Place,Sandwich Place,Coffee Shop,,,5.200000,7.966667,3,55959,35.9,3.3,80.15,16805,59760.0,34.74,62.55,2.71


### 5. Results

### 5.1. Number of Postcodes per Cluster

In [46]:
toronto_merged.groupby(COL_NAME_CLUSTER_LABELS)[COL_NAME_POSTCODE].count()

Cluster Labels
0     7
1    18
2     7
3     8
4    34
5     4
6    25
Name: Postcode, dtype: int64

### 5.2. Cluster Details

#### Let's define a function to extract the cluster data (3 tops venues and all numeric columns)

In [47]:
def extractClusterData(df, cluster_labels):
    return df.loc[
            df[COL_NAME_CLUSTER_LABELS] == cluster_labels,
            df.columns[[0, 1, 2] + list(range(6, df.shape[1]))]
        ]

In [48]:
from IPython.core.display import HTML

for cluster_label in range(0, K_CLUSTERS):
    display(HTML('<h2 style="color:darkblue;">---- Cluster {} ----</h2>'.format(cluster_label)))
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(HTML(extractClusterData(toronto_merged, cluster_label).to_html(index=False)))

Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M2H,North York,Hillcrest Village,Park,Sandwich Place,Bakery,Pizza Place,Coffee Shop,0.0,6.400000,1,24497,46.3,2.8,69.04,8700,72220.0,30.69,65.45,3.86
M2K,North York,Bayview Village,Park,Café,,,,0.0,6.680000,1,23852,40.4,2.2,57.35,10540,68015.0,34.89,59.07,6.04
M2M,North York,"Newtonbrook, Willowdale",Coffee Shop,Sandwich Place,Park,Café,,0.0,6.500000,3,32320,43.5,2.6,64.63,12400,60440.0,27.88,64.88,7.24
M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",Coffee Shop,Italian Restaurant,Sandwich Place,Restaurant,Pizza Place,0.0,6.788889,3,18241,43.8,1.9,43.15,9665,85487.0,68.52,28.13,3.34
M9A,Etobicoke,Islington Avenue,Park,Café,,,,0.0,7.700000,9,35594,43.1,2.3,61.35,15120,75907.0,56.21,41.35,2.44
M9L,North York,Humber Summit,Bakery,Pizza Place,,,,0.0,9.600000,0,11950,40.3,3.2,73.74,3745,62899.0,39.33,58.54,2.13
M9M,North York,"Emery, Humberlea",Italian Restaurant,,,,,0.0,4.700000,0,22263,37.6,3.0,77.22,7310,61884.0,40.34,57.81,1.86


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M1H,Scarborough,Cedarbrae,Bakery,Coffee Shop,,,,5.200000,3.800000,1,24383,38.1,2.7,70.77,8985,58492.0,37.27,57.95,4.77
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",Pizza Place,Park,,,,4.966667,5.400000,2,22913,44.8,2.6,71.10,8625,68550.0,58.52,40.39,1.08
M1R,Scarborough,"Maryvale, Wexford",Bar,Restaurant,Pizza Place,Café,Coffee Shop,7.440000,5.150000,3,29858,41.9,2.7,72.66,10770,60881.0,46.59,51.61,1.80
M1X,Scarborough,Upper Rouge,,,,,,6.100000,6.550000,0,15097,36.2,4.1,87.26,3650,96107.0,36.29,62.94,0.77
M3J,North York,"Northwood Park, York University",Pizza Place,Bar,Sandwich Place,Coffee Shop,,6.700000,6.300000,3,25473,34.8,2.7,72.07,9405,54031.0,36.99,56.48,6.54
M3K,North York,"CFB Toronto, Downsview East",Coffee Shop,Sandwich Place,Pizza Place,Park,,6.300000,7.550000,2,5997,42.5,2.7,70.53,2210,62635.0,43.69,54.22,2.09
M3L,North York,Downsview West,Park,Pizza Place,Coffee Shop,,,5.850000,3.500000,0,17339,41.3,3.0,77.08,5820,59669.0,38.96,59.07,1.97
M3M,North York,Downsview Central,Restaurant,,,,,5.850000,0.000000,2,24046,40.2,2.6,72.13,9235,55099.0,41.89,55.50,2.61
M4A,North York,Victoria Village,Park,Coffee Shop,,,,7.300000,6.775000,4,14443,44.2,2.3,69.39,6170,54905.0,47.10,51.64,1.26
M4H,East York,Thorncliffe Park,Coffee Shop,Sandwich Place,Restaurant,Pizza Place,Park,7.100000,6.650000,3,19688,32.4,3.0,80.26,6415,43945.0,30.05,65.51,4.44


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",Café,Hotel,Coffee Shop,Restaurant,Bar,5.800000,5.488889,49,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,Bakery,5.200000,5.488889,49,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Restaurant,Hotel,Bakery,5.200000,6.100000,49,15,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M5X,Downtown Toronto,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Restaurant,Bar,5.800000,5.610000,49,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M7A,Queen's Park,Queen's Park,Coffee Shop,Italian Restaurant,Park,Café,Sandwich Place,6.885714,5.600000,8,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M7R,Mississauga,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Pizza Place,Sandwich Place,Bakery,0.000000,6.000000,3,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,Bar,Bakery,Italian Restaurant,Pizza Place,Coffee Shop,6.650000,6.600000,6,10,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M2N,North York,Willowdale South,Coffee Shop,Pizza Place,Sandwich Place,Restaurant,Café,7.900000,7.166667,1,75897,37.0,2.2,60.75,33765,62919.0,30.84,59.40,9.76
M4Y,Downtown Toronto,Church and Wellesley,Coffee Shop,Café,Restaurant,Pizza Place,Park,7.187500,5.100000,5,30472,34.0,1.5,29.60,19665,52167.0,52.19,35.69,12.12
M5A,Downtown Toronto,"Harbourfront, Regent Park",Coffee Shop,Park,Bakery,Restaurant,Café,6.733333,6.033333,4,41078,37.0,1.7,47.20,22335,52623.0,59.00,37.84,3.16
M5V,Downtown Toronto,"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",Coffee Shop,Bar,Park,,,6.633333,8.100000,0,49195,31.4,1.6,28.91,30725,77752.0,59.27,33.30,7.43
M6H,West Toronto,"Dovercourt Village, Dufferin",Coffee Shop,Park,Bar,Bakery,Café,7.220000,5.900000,4,44950,36.4,2.3,58.68,19210,63550.0,57.31,38.90,3.80
M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Bakery,Restaurant,Café,Bar,6.725000,6.371429,10,40957,34.6,1.8,47.92,22025,54000.0,58.19,38.36,3.46
M6P,West Toronto,"High Park, The Junction South",Bar,Italian Restaurant,Café,Park,Coffee Shop,7.900000,6.060000,3,40035,37.8,2.1,55.93,18975,68983.0,64.16,32.98,2.86
M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",Café,Sandwich Place,Bakery,Italian Restaurant,Restaurant,7.500000,7.525000,4,37975,41.6,1.9,53.40,19605,61120.0,57.60,39.25,3.14


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",Bar,Italian Restaurant,,,,6.950000,4.125000,1,35626,44.0,3.1,70.14,11275,109785.0,54.12,44.89,0.99
M1N,Scarborough,"Birch Cliff, Cliffside West",Café,,,,,5.450000,5.600000,5,22136,45.2,2.4,65.99,9095,73256.0,70.67,28.45,0.88
M2L,North York,"Silver Hills, York Mills",Park,,,,,6.950000,7.000000,1,11717,45.0,2.9,69.69,3975,115493.0,47.49,49.94,2.58
M2P,North York,York Mills West,Park,,,,,7.600000,7.700000,4,7843,45.8,2.6,66.04,3020,107520.0,51.35,46.47,2.19
M3B,North York,Don Mills North,Café,,,,,6.400000,6.275000,0,13324,47.1,2.6,66.10,5000,108267.0,50.30,47.25,2.45
M4B,East York,"Parkview Hill, Woodbine Gardens",Pizza Place,Bakery,,,,6.450000,6.766667,7,18628,41.9,2.4,69.10,7600,60749.0,61.18,36.99,1.83
M4E,East Toronto,The Beaches,Bar,Sandwich Place,Coffee Shop,Bakery,Pizza Place,6.150000,6.466667,7,25044,41.9,2.3,65.30,10785,94587.0,79.13,19.88,0.99
M4G,East York,Leaside,Coffee Shop,Sandwich Place,Restaurant,Bakery,Italian Restaurant,8.466667,7.983333,3,19076,43.3,2.4,67.70,7730,115833.0,76.04,22.71,1.25
M4J,East York,East Toronto,Café,Pizza Place,Coffee Shop,Park,Bar,7.040000,6.533333,3,35738,40.8,2.4,65.92,14750,72720.0,66.35,32.47,1.18
M4K,East Toronto,"Riverdale, The Danforth West",Coffee Shop,Park,Café,Italian Restaurant,Sandwich Place,7.100000,6.090000,5,31583,41.5,2.1,60.10,14555,65948.0,67.13,31.07,1.79


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M5C,Downtown Toronto,St. James Town,Coffee Shop,Café,Hotel,Bakery,Restaurant,6.15,5.275,19,2951,36.4,1.6,30.65,1720,83872.0,60.82,35.42,3.76
M5E,Downtown Toronto,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Bakery,5.20,6.100,44,9118,34.5,1.6,27.05,5680,89969.0,57.37,36.89,5.74
M5H,Downtown Toronto,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Bar,Pizza Place,5.80,5.610,48,2005,31.7,1.6,34.18,1245,86101.0,53.89,39.64,6.48
M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Italian Restaurant,Café,Restaurant,5.80,6.100,43,14545,33.3,1.7,33.88,8645,85077.0,50.39,41.91,7.70


Postcode,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
M1B,Scarborough,"Malvern, Rouge",Coffee Shop,,,,,6.240000,5.933333,3,66108,38.2,3.3,80.10,20230,69126.0,38.21,59.92,1.87
M1E,Scarborough,"Guildwood, Morningside, West Hill",Pizza Place,Coffee Shop,,,,5.033333,5.033333,5,46943,42.2,2.7,72.57,17160,62047.0,50.74,47.79,1.47
M1G,Scarborough,Woburn,Park,Coffee Shop,,,,4.075000,5.750000,0,29690,37.2,3.0,76.57,9765,54450.0,40.54,56.03,3.43
M1J,Scarborough,Scarborough Village,Sandwich Place,Restaurant,Pizza Place,Coffee Shop,,6.400000,5.300000,1,36699,37.2,2.9,79.17,12275,54507.0,40.73,56.85,2.42
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",Coffee Shop,Sandwich Place,,,,6.460000,5.440000,4,48434,40.1,2.7,75.42,17930,53260.0,42.06,55.41,2.53
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",Bakery,Coffee Shop,Pizza Place,Park,,6.125000,6.150000,6,35081,38.0,2.8,75.55,12430,56779.0,45.30,52.20,2.50
M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford Heights",Bakery,Coffee Shop,,,,6.000000,4.500000,4,45571,39.3,2.7,70.76,16540,56661.0,38.18,57.60,4.22
M1S,Scarborough,Agincourt,Sandwich Place,Coffee Shop,,,,7.200000,7.200000,2,37769,44.3,2.9,69.90,12645,63376.0,29.60,66.17,4.23
M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Sandwich Place,Italian Restaurant,Coffee Shop,,6.360000,6.666667,2,34588,44.6,2.6,68.69,12945,53535.0,33.67,62.79,3.53
M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Pizza Place,Park,Coffee Shop,,,7.280000,7.300000,3,54680,43.0,3.4,73.65,16090,64576.0,26.36,69.64,4.00


In [49]:
cols = toronto_merged.columns[[0]+list(range(5,toronto_merged.shape[1]))]

# Averages on numeric features of each cluster
toronto_merged[COL_NAME_TRANSPORTATION_COUNT] = toronto_merged[COL_NAME_TRANSPORTATION_COUNT].astype(int)
toronto_merged[cols].groupby(COL_NAME_CLUSTER_LABELS).mean()

,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
Cluster Labels,,,,,,,,,,,,
0,0.000000,6.909841,2.428571,24102.428571,42.142857,2.571429,63.782857,9640.000000,69550.285714,42.551429,53.604286,3.844286
1,6.426481,4.830992,3.888889,18951.388889,37.816667,2.511111,67.159444,7534.722222,56140.000000,43.737778,51.050000,5.212222
2,5.076531,5.841111,30.428571,6.428571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,7.224896,6.532054,3.875000,45069.875000,36.225000,1.887500,47.798750,23288.125000,61639.250000,54.820000,39.465000,5.716250
4,7.443186,6.889511,2.500000,22639.000000,41.911765,2.338235,61.495588,9636.323529,88599.441176,65.060294,32.721765,2.217941
5,5.737500,5.771250,38.500000,7154.750000,33.975000,1.625000,31.440000,4322.500000,86254.750000,55.617500,38.465000,5.920000
6,6.014133,6.015467,3.040000,42533.920000,40.116000,2.776000,72.959600,15151.600000,60056.360000,40.668800,56.274000,3.056400


### 5.3. Visualize Clusters

In [50]:
# Create a map of Toronto with postal code locations superimposed on top

map_clusters = folium.Map(
    location=[toronto_map_center[COL_NAME_LATITUDE].values[0], toronto_map_center[COL_NAME_LONGITUDE].values[0]],
    zoom_start=11
)

# set color scheme for the clusters
x = np.arange(K_CLUSTERS)
ys = [i + x + (i*x)**2 for i in range(K_CLUSTERS)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# create feature groups, each for each cluster
feature_groups = []
for i in range(0, K_CLUSTERS):
    feature_group = folium.FeatureGroup(
        name='<span style=\\"color: {};\\">Cluster {}</span>'.format(rainbow[i], i)
    )
    feature_group.add_to(map_clusters)

    feature_groups.append(feature_group)

# add markers to the map
for postcode, lat, lon, poi, cluster in zip(
    toronto_merged[COL_NAME_POSTCODE],
    toronto_merged[COL_NAME_LATITUDE],
    toronto_merged[COL_NAME_LONGITUDE],
    toronto_merged[COL_NAME_NEIGHBOURHOOD],
    toronto_merged[COL_NAME_CLUSTER_LABELS]):
    # print(poi, lat, lon, cluster)
    label = folium.Popup(postcode + ' - ' + str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7
    ).add_to(feature_groups[cluster])

folium.map.LayerControl('topright', collapsed=False).add_to(map_clusters)

map_clusters

In [51]:
# Toronto City Hall
# 100 Queen St W, Toronto, ON M5H 2N2
toronto_merged[toronto_merged[COL_NAME_POSTCODE] == 'M5H']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,5,Café,Coffee Shop,Hotel,Bar,Pizza Place,5.8,5.61,48,2005,31.7,1.6,34.18,1245,86101.0,53.89,39.64,6.48


In [52]:
# Bridle Path, North York
# One of the most expensive neighbourhood in Toronto
# Observation: I was surprise that the Census Median Family Income was only about $58k
toronto_merged[toronto_merged[COL_NAME_POSTCODE] == 'M3C']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
27,M3C,North York,"Don Mills South, Flemingdon Park",43.7259,-79.340923,6,Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,,4.4,7.0,4,39153,40.4,2.4,68.34,15850,57275.0,38.05,58.61,3.34


In [53]:
# Yorkdale Shopping Centre
# 3401 Dufferin St, North York, ON M6A 2T9
# 1st tier shopping mall
toronto_merged[toronto_merged[COL_NAME_POSTCODE] == 'M6A']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
71,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,Restaurant,Coffee Shop,Park,,,6.5,3.3,7,21048,39.0,2.5,73.38,8075,53933.0,46.96,49.78,3.26


In [54]:
# Dufferin Mall
# 900 Dufferin St, Toronto, ON M6H 4A9
# 2nd tier shopping mall (South of Yorkdale Shopping Centre)
toronto_merged[toronto_merged[COL_NAME_POSTCODE] == 'M6H']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
76,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,3,Coffee Shop,Park,Bar,Bakery,Café,7.22,5.9,4,44950,36.4,2.3,58.68,19210,63550.0,57.31,38.9,3.8


### 6. Data Analysis

There are clusters of neighbourhoods in Toronto that have salient charateristics:

#### Cluster 0: Good Schools and Families with Children
- Top venues: Eateries and parks
- Good secondary schools but no elementary school rating found
- Many families with children (63.7%)
- Medium population density

#### Cluster 1: Good Elementary Schools and Young Families with Children
- Top venues: Eateries and parks
- Good elementary schools but poor secondary school ratings
- Young families in the neighbourhoods (Census Median Age < 40)
- Many families with children (67%)
- Low population density
- Look like these were new neighbourhoods in development

#### Cluster 2: Postcodes with insufficient census data
- There were no census data (zero most of the cases) in these postcode areas because the census population size was very small (less than 15)
- No characteristics could be determined

#### Cluster 3: Downtown Toronto, Average Schools, and High Population Density
- Top venues: Eateries (coffee, pizza, and restaurants) and hotels
- Household Size was small, at 1.89 persons per household
- Both Elementary and Secondary School Ratings were average to good
- High Population Density due to the highest Census Population Size with small Household Size, most likely high rise apartments

#### Cluster 4: Good Schools, High Family Income, and Non-Immigrants
- Top venues: Eateries and parks
- Good elementary and secondary school ratings
- High family income
- Predominantly non-immigrants (Census non-Immigrants = 65%)
- Older neighbourhoods located South of Highway 401

#### Cluster 5: Downtown Toronto, High Family Income, and Young Families
- Top venues: Eateries and hotels
- Schools were poor with both ratings below 6
- Household Size was small, at 1.62 persons per houshold, which was consistent with the low Family With Children (31%) 
- High Census Median Family Income (over \$82k)
- Look like mostly young (Census Median Age = 34) professional couples without children lived in these neighbourhood

#### Cluster 6: Newer Neighbourhoods with High Population Density and Families with Children
- To venues: Eateries and parks
- Schools were average
- High population density with large Census Population Size (over 42k persons) and the highest Census Average Household Size (2.78 persons per household)
- 73% families with children, significantly higher than other clusters

#### Additional Observations in General about Toronto
- It seems elementary and secondary school ratings were poor in general, mostly 6 or below
- Non-immigrants and immigrants ratios was close to 50:50 in average
- Household size was small with average below 3

### 7. Discussion

There were few issues and difficulties on data quality and data acquisition that could impact our data analysis and possibly our results.

### 7.1. Issues On Data Quality

We can observe that the Avg Rating Elementary Schools for Cluster 0 is zero.  This could be:
    1. There were no elementary schools in postcode areas of this cluster; and/or
    2. Foursquare did not have complete school information; and/or
    3. The elementary schools in the area were not listed in the data source Fraser Institute
Taking the postcode M2K in Cluster 0, there was an elementary school "Pineway Public School" from Foursquare; however, Fraser did not have school rating for this school.

On alternative could be assigned the same value from Avg Rating Secondary Schools as approximation for the missing Avg Rating Elementary Schools.

Also, the transportation information from Foursquare was also incomplete in many areas.  Using the same postcode M2K of Bayview Village in North York as an example, there was only one bus stop in 10 minute walking distance from Foursquare.  However, looking from Toronto Transit Commission (TTC) web site, there were many TTC bus stops in the area.

Foursquare venues data were leaning toward eateries and places where people would spend their leisure times.  It provided poor information on transportations for example.

In addition, although 2016 Census data from Statistic Canada have a lot of details but these data are already at least 3 years old.  Demographic information in neighbourhoods with high immigrants and non-permanent residents and with busy recent real estate development would make the census data inaccurate until next Census in 2021.

In [56]:
toronto_merged[toronto_merged[COL_NAME_POSTCODE] == 'M2K']

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Avg Rating Elementary Schools,Avg Rating High Schools,Transportation Count,Census Population Size,Census Median Age,Census Average Household Size,Census Households With Children (%),Census Number of Households,Census Median Family Income,Census Non Immigrants (%),Census Immigrants (%),Census Non Permanent Residents (%)
19,M2K,North York,Bayview Village,43.786947,-79.385975,0,Park,Café,,,,0.0,6.68,1,23852,40.4,2.2,57.35,10540,68015.0,34.89,59.07,6.04


### 7.2. Difficulties On Data Acquisition

Originally, I was looking to scrap HTML from HouseSigma for median pricing for residential properties; however this could not be done because the listed communities could not be matched to the neighbourhoods from Wikipedia.

I could not find another data source for Toronto residential pricing; therefore, one of the most important features, house pricing, could not be included in the analysis. 

### 8. Conclusion

Answering the question stated of this study: Which neighbourhoods have the similar features from where a family with young children can purchase a detached residential property?

We grouped the neighbourhoods into 7 clusters using K-Means clustering algorithm; having one of them (with 7 postcode areas) without enough census and school data.  A family can determine which features are the most important and then search the neighbourhoods of the clusters which has the features the family is looking for.

Obviously, there are many improvements can be made to improve the data quality to get better clustering.  As a matter of fact, it is expected that using local transit agencies data can provide better data than Foursquare.  For this study, using data from TTC (Toronto Transit Commission) and Go Transit can get much accurate information on bus and train stops at any location.

However, there are "human factors" that cannot include in "mathematical analysis" which sway the house purchasing decision.  Examples are: nearness to other family members or friends, closeness to employment sites, ambient of the neighbourhood (such as country style), ravines or old large trees, etc.